<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

First we need to install some libraries

In [1]:
#!pip install bs4
#!pip install geocoder
!pip install lxml

Then we need to import the following libraries

In [2]:
import pandas as pd
import numpy as np
import lxml

Let's first prepare the Toronto data

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = pd.read_html(url)
df = data[0]

df['Borough'][df['Borough']=='Not assigned']=np.NaN
df.dropna(axis=0,inplace=True)

df=df.groupby(['Postcode', 'Borough']).agg({'Neighbourhood' : ','.join})

df.reset_index(inplace=True)

df[df['Neighbourhood']=='Not assigned']

df['Neighbourhood'][df['Neighbourhood']=='Not assigned']=df['Borough'][df['Neighbourhood']=='Not assigned']

df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Next let's get the coordinate data _(geocoder didn't work)_

In [4]:
coords = pd.read_csv('https://cocl.us/Geospatial_data')

coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Next let's rename the coordinate Postal Code column to Postcode

In [5]:
coords.rename(columns={'Postal Code':'Postcode'},inplace=True)

coords.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now let's merge the two dataframes

In [6]:
pd.merge(df, coords, left_on=  ['Postcode'],
            right_on= ['Postcode'], 
            how = 'left')

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
